In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import networkx as nx

import sys
sys.path.append('../src')

from simulated_annealing import simulated_annealing
from simulated_annealing import hamiltonian_objective_function
from simulated_annealing import flip_step_function
from simulated_annealing import single_pass_optimize
from simulated_annealing import multi_pass_optimize
from simulated_annealing import gradient_descent

In [3]:
# Load the graph from the GraphML file
G = nx.read_graphml('out\\2016_2019.graphml')

# Get the adjacency matrix
adj_matrix = nx.adjacency_matrix(G).todense()

# Convert the adjacency matrix to a numpy array (optional)
couplings = np.array(adj_matrix)
couplings_from_csv = np.genfromtxt('out\\correlation_matrix_2016_2019.graphml.csv', delimiter=',')  # Adjust parameters as needed
couplings_from_csv = couplings_from_csv - np.identity(couplings_from_csv.shape[0])

In [4]:
print((couplings_from_csv == couplings).all())

True


In [5]:
def evaluate_minima(minima):
    costs = hamiltonian_objective_function(minima, couplings)
    unique_minima, counts = np.unique(minima, axis=0, return_counts=True)
    num_found = unique_minima.shape[0]
    top_five_minima = counts[(-counts).argsort()][:5]

    print("mean cost", costs.mean(), "min cost", costs.min())
    print("number found", num_found)
    print("top five", top_five_minima)
    print(counts.sum())

In [6]:
# find minima with gradient descent
initial_vectors = np.random.choice([-1, 0, 1], size=(1000, 101))
mp_minima, num_changed = multi_pass_optimize(initial_vectors, couplings, max_iterations=int(1e4))
print(num_changed)

0


In [7]:
evaluate_minima(mp_minima)

mean cost -20.26444925207737 min cost -23.940733741828687
number found 965
top five [3 2 2 2 2]
1000


In [178]:
# find minima with gradient descent
initial_vectors_gd = np.random.choice([-1, 0, 1], size=(1000, 101))
gd_minima = gradient_descent(initial_vectors_gd, 0.1, couplings, diff_threshold=0.01, max_iterations=int(1e5))

In [179]:
evaluate_minima(gd_minima)

mean cost -20.590500837658624 min cost -23.944227654207438
number found 1000
top five [1 1 1 1 1]
1000


In [180]:
# find minima with simulated annealing
initial_vectors = np.random.choice([-1, 0, 1], size=(1000, 101))
sa_minima, costs = simulated_annealing(initial_vectors, 100, 0.99, int(1e5), 
                              lambda vecs: hamiltonian_objective_function(vecs, couplings),
                              lambda vecs: flip_step_function(vecs, num_flips=1))

c:\Users\vicvi\BeliefNetworkEvo\nbk\../src\simulated_annealing.py:65: RuntimeWarning: overflow encountered in exp
  acceptance_prob_vector = np.where(new_cost_vector < old_cost_vector, 1, np.exp((old_cost_vector - new_cost_vector) / temperature))


In [181]:
evaluate_minima(sa_minima)

mean cost -20.93212400741058 min cost -23.945401174822344
number found 1000
top five [1 1 1 1 1]
1000


In [182]:
mixed_minima, num_changed = multi_pass_optimize(sa_minima, couplings, random_order=True)

In [183]:
evaluate_minima(mixed_minima)

mean cost -20.99256975265968 min cost -23.945401174822344
number found 929
top five [3 3 3 3 3]
1000


In [71]:
# Create a graph
G = nx.Graph()

# Add nodes to the graph
num_nodes = 101
G.add_nodes_from(range(num_nodes))

# Add edges to the graph (example: add an edge between each pair of nodes)
for i in range(num_nodes):
    for j in range(i + 1, num_nodes):
        G.add_edge(i, j)

# Assign a real value between -1 and 1 to each node
for node in G.nodes:
    G.nodes[node]['value'] = random.uniform(-1, 1)

# Export the graph to a GraphML file
nx.write_graphml(G, 'graph.graphml')

print("Graph exported to 'graph.graphml'")

158

In [203]:
for u, v, d in G.edges(data=True):
    d['weight'] = abs(d['weight'])

In [204]:
sample = np.random.choice(np.arange(1000), replace=False, size=(10,))

for j, index in enumerate(sample):
    vector = mp_minima[index,:]

    for i, node in enumerate(G.nodes):
        G.nodes[node]['value'] = vector[i]

    nx.write_graphml(G, f'out\\valley_2016_2019_{j}.graphml')

In [198]:
np.where(couplings[2,:] > 0)

(array([18], dtype=int64),)

In [11]:
var_list = list(G.nodes)

In [8]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(mp_minima)

PCA()

In [17]:
above_average = (pca.components_ > 0.08)

In [18]:
for i in range(10):
    comp = above_average[i,:]
    var_indices = np.where(comp)[0]
    rel_nodes = [node for j, node in enumerate(var_list) if j in var_indices]
    print(rel_nodes)

['NATCRIME', 'NATARMS', 'EQWLTH', 'GUNLAW', 'GRASS', 'PRAYER', 'AFFRMACT', 'SEXEDUC', 'LETDIE1', 'SUICIDE1', 'RACDIF1', 'RACDIF3', 'HELPPOOR']
['LIBATH', 'LIBRAC', 'COLCOM', 'LIBCOM', 'LIBMIL', 'LIBHOMO', 'LIBMSLM', 'PREMARSX', 'HOMOSEX', 'PORNLAW', 'SPANKING']
['SPKATH', 'COLATH', 'SPKRAC', 'COLRAC', 'SPKCOM', 'SPKMIL', 'COLMIL', 'SPKMSLM', 'COLMSLM', 'ATTEND', 'RELITEN', 'POSTLIFE', 'CONFINAN', 'CONBUS', 'CONCLERG', 'CONEDUC', 'CONFED', 'CONLABOR', 'CONMEDIC', 'CONJUDGE', 'CONSCI', 'CONLEGIS', 'CONARMY', 'PREMARSX', 'TEENSEX', 'XMARSEX', 'HOMOSEX', 'PORNLAW', 'SPANKING']
['PREMARSX', 'TEENSEX', 'XMARSEX', 'HOMOSEX', 'PORNLAW']
['NATHEAL', 'NATCITY', 'NATCRIME', 'NATDRUG', 'NATEDUC', 'NATRACE', 'NATARMS', 'NATSOC', 'EQWLTH', 'GUNLAW', 'WRKWAYUP', 'CONPRESS', 'CONTV', 'TVHOURS', 'RACDIF2', 'RACDIF4', 'HELPPOOR']
['LIBHOMO', 'ATTEND', 'RELITEN', 'POSTLIFE', 'FEPOL', 'ABDEFECT', 'ABNOMORE', 'ABHLTH', 'ABPOOR', 'ABRAPE', 'ABSINGLE', 'ABANY', 'PREMARSX', 'TEENSEX', 'XMARSEX', 'HOMOSEX', 'P